In [1]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("movies.csv")

# Drop the specified columns
columns_to_drop = ['summary', 'poster_link', 'metadata']
df.drop(columns=columns_to_drop, inplace=True)

# Optionally, you can save the modified DataFrame to a new CSV file
# df.to_csv("modified_movies.csv", index=False)

df.head()

,imdb_id,title,date,genres,runtime,rating,votes,budget,revenue,language,production
0,114709,Toy Story (1995),1995-10-30T00:00:00Z,Animation,81,8.3,5415,30000000,373554033,English,Pixar Animation Studios
1,113497,Jumanji (1995),1995-12-15T00:00:00Z,Action,104,6.9,2413,65000000,262797249,English,TriStar Pictures
2,113228,Grumpier Old Men (1995),1995-12-22T00:00:00Z,Comedy,101,6.6,92,0,0,English,Warner Bros.
3,113041,Father of the Bride Part II (1995),1995-02-10T00:00:00Z,Comedy,106,5.9,173,0,76578911,English,Sandollar Productions
4,113277,Heat (1995),1995-12-15T00:00:00Z,Action,170,8.2,1886,60000000,187436818,English,Regency Enterprises


In [2]:
df.to_csv("metadata.csv", index=False)

In [3]:
print(len(df))

8646


In [16]:
from dotenv import load_dotenv, find_dotenv
import json
from openai import OpenAI
import openai
import os

load_dotenv(find_dotenv())
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY'),
)

def parse_user_query(user_query):

    # tools = [{"type": "PythonAstREPLTool"}]
    
    prompt_template = f"""
        Your task is to parse the following query '{user_query}' provided by a user and provide only the SQL output that can run on the pandas Dataframe tabled named 'df' and query movies that matches the criteria mentioned in the user query. Make sure to use 'FROM df' in the SQL query.
        The explanation of each column in the pandas Dataframe is as follows:
        "title": The title of a film in string format.
        "date": The release date of the film formatted in ISO 8601 format.
        "genres": The genre of the film in string format.
        "runtime": The runtime of the movie in munutes in a whole number integer format.
        "votes": The voting count of a film in a whole number integer format.
        "budget": The dollar budget of a film in a whole number integer format.
        "revenue": The dollar revenue of a film in a whole number integer format.
        "language": The language of the film in string format.
        "production": The production company of the film in string format.
        The expected example output is the follwoing:
        'SELECT * 
        FROM df
        WHERE date >= '2008-01-01' AND genres LIKE '%Animation%';'
    """

    messages = [{
            "role": "system",
            "content": "Please generate output in SQL format exclusively, avoiding any additional text or explanations.",
        },
        {
            "role": "user",
            "content": prompt_template
        }
    ]

    stream = client.chat.completions.create(
        # tools=tools,
        model="gpt-3.5-turbo-0125",
        messages=messages,
        max_tokens=500,
        temperature=0.5,
        frequency_penalty=0,
        presence_penalty=0,
        # response_format={ "type": "json_object" }
    )
    # return json.loads(stream.choices[0].message.content)
    return stream.choices[0].message.content


In [17]:
response = parse_user_query('2008年以降に作られたアニメーション映画')
print(response)

SELECT * 
FROM df
WHERE date >= '2008-01-01' AND genres LIKE '%Animation%';


In [15]:
from pandasql import sqldf
result = sqldf(response)
print(result)

     imdb_id                                            title  \
0     451079                       Horton Hears a Who! (2008)   
1     902272          Justice League: The New Frontier (2008)   
2     441773                             Kung Fu Panda (2008)   
3     910970                                   WALLåáE (2008)   
4    1054485  Futurama: The Beast with a Billion Backs (2008)   
..       ...                                              ...   
281  5311514                             Kimi no na wa (2016)   
282  4624424                                    Storks (2016)   
283  2321405                     My Life as a Zucchini (2016)   
284  3682556                               The Present (2014)   
285  1679335                                    Trolls (2016)   

                     date     genres  runtime  rating  votes     budget  \
0    2008-03-03T00:00:00Z  Animation       88     6.9    927   85000000   
1    2008-02-26T00:00:00Z  Animation       75     7.0    115    35000

In [18]:
result_dict_list = result.to_dict(orient='records')
print(result_dict_list)

[{'imdb_id': 451079, 'title': 'Horton Hears a Who! (2008)', 'date': '2008-03-03T00:00:00Z', 'genres': 'Animation', 'runtime': 88, 'rating': 6.9, 'votes': 927, 'budget': 85000000, 'revenue': 297138014, 'language': 'English', 'production': 'Twentieth Century Fox Film Corporation'}, {'imdb_id': 902272, 'title': 'Justice League: The New Frontier (2008)', 'date': '2008-02-26T00:00:00Z', 'genres': 'Animation', 'runtime': 75, 'rating': 7.0, 'votes': 115, 'budget': 3500000, 'revenue': 5231128, 'language': 'English', 'production': 'Warner Bros. Entertainment'}, {'imdb_id': 441773, 'title': 'Kung Fu Panda (2008)', 'date': '2008-06-04T00:00:00Z', 'genres': 'Animation', 'runtime': 90, 'rating': 7.6, 'votes': 3231, 'budget': 130000000, 'revenue': 631744560, 'language': 'English', 'production': 'DreamWorks Animation'}, {'imdb_id': 910970, 'title': 'WALLåáE (2008)', 'date': '2008-06-22T00:00:00Z', 'genres': 'Animation', 'runtime': 98, 'rating': 8.4, 'votes': 6439, 'budget': 180000000, 'revenue': 5213

In [33]:
from langchain.chat_models import ChatOpenAI
# from langchain.tools import Tool
from langchain.agents import initialize_agent
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools import PythonREPLTool
# from langchain.tools.python.tool import PythonAstREPLTool
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

llm = ChatOpenAI(    
    openai_api_key=os.getenv('OPENAI_API_KEY'), 
    model_name="gpt-3.5-turbo-0125", 
    temperature=0.0
)
python_agent = create_python_agent(llm, tool=PythonREPLTool(), verbose=True)

python = PythonREPLTool(locals={"df": df})

sql_query = response

# llm_chain = LLMChain(
#     llm=llm,
#     prompt=PromptTemplate.from_template(prompt_template)
# )

# def generate_js(sql_query):

df_columns = df.columns.to_list()

# prompt_template = f"""
#     Your task is to generate interactive and visually appealing Chart.js code based on a provided string of SQL query {sql_query}.
#     The dataset that you can run the SQL query using the PythonAgent function is provided as a tool. Based on the queried results and values, you are expected to further analyze the result to determine the appropriate type of chart (e.g., bar, line, pie) that best represents the data.
#     Only return the Chart.js settings code in json format. The expected example output is the follwoing:
#     '{{ type: 'pie', data: {{ labels: ['Label 1', 'Label 2', 'Label 3', 'Label 4', 'Label 5'], datasets: [ {{ data: [30, 20, 15, 10, 25], backgroundColor: [ 'rgba(255, 99, 132, 0.7)', 'rgba(54, 162, 235, 0.7)', 'rgba(255, 206, 86, 0.7)', 'rgba(75, 192, 192, 0.7)', 'rgba(153, 102, 255, 0.7)', ], }}, ], }}, options: {{ responsive: true, maintainAspectRatio: false, aspectRatio: 1, plugins: {{ legend: {{ display: true, position: 'top', }}, }}, }}, }})'
#     Nothing else. YOU ARE TO ONLY RETURN CHART.JS CODE formatted correctly.
# """

# messages = [{
#     "role": "system",
#     "content": "Please generate the Chart.JS code in JSON format exclusively, avoiding any additional text or explanations.",
# },
# {
#     "role": "user",
#     "content": prompt_template
# }]

tools=[
    Tool(
        name="PythonAgent",
        func=python_agent.run,
        description=f"""
        When you need to see the queried results and values from the data stored in pandas dataframe 'df'. 
        Run python pandas operations, such as 'sqldf({sql_query})' on 'df' to help you get the right answer. 
        The 'df' has the following columns: {df_columns}
        """,
    ),
],

# stream = client.chat.completions.create(
#     tools=tools,
#     model="gpt-3.5-turbo-0125",
#     messages=messages,
#     max_tokens=500,
#     temperature=0.5,
#     frequency_penalty=0,
#     presence_penalty=0,
#     response_format={ "type": "json_object" }
# )
# return json.loads(stream.choices[0].message.content)
# return stream.choices[0].message.content

agent_kwargs = {
    'prefix': f"""
    Your task is to generate interactive and visually appealing Chart.js code based on a provided string of SQL query {sql_query}.
    The dataset that you can run the SQL query using the PythonAgent function is provided as a tool. Based on the queried results and values, you are expected to further analyze the result to determine the appropriate type of chart (e.g., bar, line, pie) that best represents the data.
    Only return the Chart.js settings code in json format. The expected example output is the follwoing:
    '{{ type: 'pie', data: {{ labels: ['Label 1', 'Label 2', 'Label 3', 'Label 4', 'Label 5'], datasets: [ {{ data: [30, 20, 15, 10, 25], backgroundColor: [ 'rgba(255, 99, 132, 0.7)', 'rgba(54, 162, 235, 0.7)', 'rgba(255, 206, 86, 0.7)', 'rgba(75, 192, 192, 0.7)', 'rgba(153, 102, 255, 0.7)', ], }}, ], }}, options: {{ responsive: true, maintainAspectRatio: false, aspectRatio: 1, plugins: {{ legend: {{ display: true, position: 'top', }}, }}, }}, }})'
    Nothing else. YOU ARE TO ONLY RETURN CHART.JS CODE formatted correctly.
    You have access to the following tools:
    """
}


# initialize the LLM agent
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True, 
    agent_kwargs=agent_kwargs
)
        # define q and a function for frontend
def get_response(user_input):
    response = agent.run(user_input)
    return response

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  def emit_warning() -> None:


AttributeError: 'list' object has no attribute 'is_single_input'

In [50]:
def get_response(user_input):
    python_agent = create_python_agent(llm, tool=PythonREPLTool(), verbose=True)
    python = PythonREPLTool(locals={"df": df})
    sql_query = user_input
    df_columns = df.columns.to_list()
    
    tools = [Tool(
        name="PythonAgent",
        func=python_agent.run,
        description=f"""
        When you need to see the queried results and values from the data stored in pandas dataframe 'df'. 
        Run python pandas operations, such as 'sqldf({sql_query})' on 'df' to help you get the right answer. 
        The 'df' has the following columns: {df_columns}
        """,
    )]
    
    agent_kwargs = {
        'prefix': f"""
        Your task is to generate interactive and visually appealing Chart.js code based on a provided string of SQL query {sql_query}.
        The dataset that you can run the SQL query using the PythonAgent function is provided as a tool. Based on the queried results and values, you are expected to further analyze the result to determine the appropriate type of chart (e.g., bar, line, pie) that best represents the data.
        Only return the Chart.js settings code in json format. The expected example output is the follwoing:
        '{{ type: 'pie', data: {{ labels: ['Label 1', 'Label 2', 'Label 3', 'Label 4', 'Label 5'], datasets: [ {{ data: [30, 20, 15, 10, 25], backgroundColor: [ 'rgba(255, 99, 132, 0.7)', 'rgba(54, 162, 235, 0.7)', 'rgba(255, 206, 86, 0.7)', 'rgba(75, 192, 192, 0.7)', 'rgba(153, 102, 255, 0.7)', ], }}, ], }}, options: {{ responsive: true, maintainAspectRatio: false, aspectRatio: 1, plugins: {{ legend: {{ display: true, position: 'top', }}, }}, }}, }})'
        Nothing else. YOU ARE TO ONLY RETURN CHART.JS CODE formatted correctly.
        You have access to the following tools:
        """
    }
    
    agent = initialize_agent(
        tools, 
        llm, 
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
        verbose=True, 
        agent_kwargs=agent_kwargs
    )
    response = agent.run(input=user_input)  # Provide the required input key
    return response

In [51]:
print(response)

SELECT * 
FROM df
WHERE date >= '2008-01-01' AND genres LIKE '%Animation%';


In [52]:
output = get_response({"input": response})



> Entering new AgentExecutor chain...


ValueError: Missing some input keys: {' type', "'input'"}

In [56]:
query = """
    SELECT genres, revenue
    FROM df
    WHERE date >= '2008-01-01' AND genres LIKE '%Animation%';
    """
result = sqldf(query)
print(result)

        genres    revenue
0    Animation  297138014
1    Animation    5231128
2    Animation  631744560
3    Animation  521311860
4    Animation          0
..         ...        ...
281  Animation  355298270
282  Animation  182379278
283  Animation     292279
284  Animation          0
285  Animation  346864462

[286 rows x 2 columns]
